* Run SalishSeaCast with different winds
* Run WaveWatchIII using results from SalishSeaCast 
* Run MOHID using results of SalishSeaCast and WaveWatchIII

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import netCDF4 as nc
import seaborn as sns
import matplotlib.colors as mcolors
import glob
import os
import xarray as xr
import datetime
from salishsea_tools import viz_tools, tidetools, geo_tools, gsw_calls, wind_tools
import pickle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

Run SalisihSeaCast from 01jan20 to 11jan20. This gives us three days of spinup and eight days of model results in order to make seven days of forcing files for mohid.

Using SOG (-123.67, 49.21) as a reference, we choose the dates

* low 350 deg winds: 2017-08-05
* high 350 deg winds: 2016-05-04
* low 150 deg winds: 2017-05-11
* high 150 deg winds: 2017-11-13

We first take wind files from the date range of our choice ( -3 days for spinup) , replace the times, and then save the file wth the appropriate name.

In [ ]:
files = np.sort(glob.glob('/ocean/vdo/MEOPAR/fake_winds/ops_y2017m11*'))[9:20]
day = 1
for file in files:
    f = xr.open_dataset(file)
    f = f.assign_coords(time_counter=[datetime.datetime(2020,1,day) + datetime.timedelta(hours = i) for i in range(24)])
    f.to_netcdf('/ocean/vdo/MEOPAR/fake_winds/10nov17/ops_y2020m01d' +f'{day:02}' +  '.nc')
    day = day + 1

In [ ]:
files = np.sort(glob.glob('/ocean/vdo/MEOPAR/fake_winds/ops_y2017m05*'))[7:18]
day = 1
for file in files:
    f = xr.open_dataset(file)
    f = f.assign_coords(time_counter=[datetime.datetime(2020,1,day) + datetime.timedelta(hours = i) for i in range(24)])
    f.to_netcdf('/ocean/vdo/MEOPAR/fake_winds/08may17/ops_y2020m01d' +f'{day:02}' +  '.nc')
    day = day + 1

In [ ]:
files = np.sort(glob.glob('/ocean/vdo/MEOPAR/fake_winds/ops_y2016m05*'))[:11]
day = 1
for file in files:
    f = xr.open_dataset(file)
    f = f.assign_coords(time_counter=[datetime.datetime(2020,1,day) + datetime.timedelta(hours = i) for i in range(24)])
    f.to_netcdf('/ocean/vdo/MEOPAR/fake_winds/01may16/ops_y2020m01d' +f'{day:02}' +  '.nc')
    day = day + 1

In [ ]:
files = np.sort(glob.glob('/ocean/vdo/MEOPAR/fake_winds/ops_y2017m08*'))[1:12]
day = 1
for file in files:
    f = xr.open_dataset(file)
    f = f.assign_coords(time_counter=[datetime.datetime(2020,1,day) + datetime.timedelta(hours = i) for i in range(24)])
    f.to_netcdf('/ocean/vdo/MEOPAR/fake_winds/02aug17/ops_y2020m01d' +f'{day:02}' +  '.nc')
    day = day + 1

We then run SalishSeaCast making sure to remember to edit the yaml file to point to the correct modified wind files.

Once SalishSeaCast is finished running, we need to nowcast-ize the results files in order to make wavewatch and mohid forcing files. This needs to be done on Skookum in the salishsea-nowcast eniviroment. We need to edit the yaml file to point the results archive : hindcast key to the directory of our SalishSeaCast results.

`python -m nowcast.workers.split_results --debug config_file hindcast 2020-01-01`

We now need to make the forcing files to run wavewatch. We need to remember to edit the yaml file to point to the correct modified wind files and to point to a desired results directory. We also need to edit the symlink `/results/SalishSea/nowcast` as we go through each date range.  We run this command from 01jan20 to 11jan20

`python -m nowcast.workers.make_ww3_wind_file SalishSeaNowcast/config/wwatch3-forcing.yaml --debug salish nowcast --run-date 2020-01-01`

`python -m nowcast.workers.make_ww3_wind_file SalishSeaNowcast/config/wwatch3-forcing.yaml --debug salish nowcast --run-date 2020-01-01`

We then transfer the wavewatch forcing to Cedar and run wavewatch. We need to remember to edit the yaml fle to point to the correct currents and wind files. We can run all 11 days with one command. It takes ~20 minutes to run one model day. 

`wwatch3 run --start-date 2020-01-01 --n-days 11 yaml_file 5:00:00 RESULTS_DIR`

We then make the mohid forcing files using make-hdf5. We need to remember to edit the yaml file to point to the correct winds, wavewatch, and nemo files. It takes ~35 min to make 7+1 days of forcing. From there we can run mohid. We need to remember to edit the yaml file to point to the forcng files that we just created. 